In [1]:
import os
%pwd


's:\\OneDrive\\Documents\\Chicken-disease-classifier--Using-MLOPS-DVC-Pipeline\\research'

In [2]:
os.chdir("../")
%pwd

's:\\OneDrive\\Documents\\Chicken-disease-classifier--Using-MLOPS-DVC-Pipeline'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
                 
                 self.config = read_yaml(config_filepath)
                 self.params = read_yaml(params_filepath)

                 create_directories([self.config.artifacts_root])
                
    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion
            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                    root_dir = config.root_dir,
                    source_URL= config.source_URL,
                    local_data_file= config.local_data_file,
                    unzip_dir= config.unzip_dir
            )
            return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import  get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} Download with following headers: {headers}")
        else:
            logger.info(f"{get_size(Path(self.config.local_data_file))} already exists")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-18 03:35:33,144: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-18 03:35:33,153: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-18 03:35:33,157: INFO: common: directory: artifacts created successfully]
[2023-08-18 03:35:33,159: INFO: common: directory: artifacts/data_ingestion created successfully]


[2023-08-18 03:35:47,938: INFO: 1118222499: artifacts/data_ingestion/data.zip Download with following headers: Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8AA0:3B34A1:215F52:3012AB:64DF4974
Accept-Ranges: bytes
Date: Fri, 18 Aug 2023 10:35:34 GMT
Via: 1.1 varnish
X-Served-By: cache-del21727-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1692354933.089058,VS0,VE1043
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 790d7f4e1e75f74f55a63595184a2a1bf787534f
Expires: Fri, 18 Aug 2023 10:40:34 GMT
Source-Age: 1

]
